In [1]:
import pandas as pd
from datetime import date, timedelta
import plotly.express as px
from typing import Dict, Generator, Tuple
from scipy.stats import binom, norm
import lsqfit
import gvar as gv
import numpy as np
import plotly.graph_objects as go

In [2]:
URLS = {
    "case_hosp_death": "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/case-hosp-death.csv",
    "testing": "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/testing.csv",
    "by_age": "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/by-age.csv",
}

In [3]:
pd.read_csv(URLS["by_age"])

,AGE_GROUP,COVID_CASE_RATE,HOSPITALIZED_CASE_RATE,DEATH_RATE
0,0-17 years,47.25,4.81,0.06
1,18-44 years,556.94,50.78,2.31
2,45-64 years,783.25,177.91,16.20
3,65-75 years,798.72,302.38,47.77
4,75 and older years,791.26,403.86,114.81
5,Citywide total,544.21,116.39,16.36


In [243]:
BIN_SIZE = 2 # in days
three_days_ago = date.today() - timedelta(days=3)

nyc_hosp_df = (
    pd.read_csv(URLS["case_hosp_death"], parse_dates=["DATE_OF_INTEREST"])
    .fillna(0)
    .astype(
        dtype={
            "NEW_COVID_CASE_COUNT": int,
            "HOSPITALIZED_CASE_COUNT": int,
            "DEATH_COUNT": int,
        },
    )
    .rename(
        columns={
            "DATE_OF_INTEREST": "date",
            "NEW_COVID_CASE_COUNT": "new_cases",
            "HOSPITALIZED_CASE_COUNT": "hospitalized",
            "DEATH_COUNT": "death",
        }
    )
    .set_index("date")
    .loc[:three_days_ago]
)

nyc_hosp_df["cases"] = nyc_hosp_df.new_cases.cumsum()
for key in ["hospitalized", "death"]:
    nyc_hosp_df[f"new_{key}"] = nyc_hosp_df[key] - nyc_hosp_df[key].shift(
        1, fill_value=0
    )

nyc_hosp_df = nyc_hosp_df.resample(f"{BIN_SIZE}D").mean()
nyc_hosp_df

,new_cases,hospitalized,death,cases,new_hospitalized,new_death
date,,,,,,
2020-03-02,1.5,2.0,0.0,2.0,1.0,0.0
2020-03-04,5.5,4.0,0.0,13.0,1.0,0.0
2020-03-06,10.5,7.0,0.0,28.5,2.5,0.0
2020-03-08,36.5,18.0,0.0,81.5,7.5,0.0
2020-03-10,113.0,46.5,0.5,256.0,15.5,0.5
2020-03-12,482.5,91.0,0.0,994.0,33.0,-0.5
2020-03-14,815.5,166.5,3.5,2428.0,33.5,2.5
2020-03-16,2169.5,312.0,8.0,6112.0,68.5,1.0
2020-03-18,3088.5,434.0,22.0,11730.5,76.0,8.0


In [244]:
df = (
    nyc_hosp_df.drop(columns=[col for col in nyc_hosp_df.columns if "new" in col])
    .stack()
    .reset_index()
    .rename(columns={"level_1": "kind", 0: "value"})
)

fig = px.scatter(
    df,
    x="date",
    y="value",
    color="kind",
    # marginal_y="histogram",
    log_y=True,
)
fig.show()

In [247]:
P0 = 0.2
P_HOSP = 0.05


case = nyc_hosp_df["cases"].values
case = gv.gvar(case, np.sqrt(case * (1 - P0)))


fig = go.Figure()


for dt in range(0, 6):

    hosp = nyc_hosp_df["hospitalized"].shift(dt).values
    hosp = gv.gvar(hosp, np.sqrt(hosp * (1 - P0 * P_HOSP)))

    yy = hosp / case * 100

    fig.add_trace(
        go.Scatter(
            x=nyc_hosp_df.index,
            y=gv.mean(yy),
            error_y_array=gv.sdev(yy),
            #mode="lines",
            name=rf"$\Delta t = {dt*BIN_SIZE} \, \text{{days}}$",
        )
    )

fig.update_layout(
    yaxis_type="log", title_text="Hospitalization vs known cases $H(t+\Delta t)/I(t)$",
    yaxis_title="Ratio in %",
    xaxis_title="Date",
)
fig.show()

In [192]:
df = nyc_hosp_df.stack().reset_index().rename(columns={"level_1": "kind", 0: "value"})

fig = px.scatter(
    df,
    x="date",
    y="value",
    color="kind",
    # marginal_y="histogram",
    log_y=True,
)
fig.show()

In [6]:
df = (
    nyc_hosp_df.drop(
        columns=[col for col in nyc_hosp_df.columns if not "new" in col] + ["new_cases"]
    )
    .stack()
    .reset_index()
    .rename(columns={"level_1": "kind", 0: "value"})
)

fig = px.scatter(
    df,
    x="date",
    y="value",
    color="kind",
    marginal_y="box",
    # log_y=True,
)
fig.show()

In [32]:
def logistic(x, L: float = 1, x0: float = 0, k: float = 0) -> float:
    denom = 1 + gv.exp(-k * (x - x0))
    return L / denom


def sir(
    s: float, i: float, r: float, beta: float, gamma: float, n: float
) -> Tuple[float, float, float]:
    """The SIR model, one time step."""
    s_n = (-beta * s * i) + s
    i_n = (beta * s * i - gamma * i) + i
    r_n = gamma * i + r
    if s_n < 0.0:
        s_n = 0.0
    if i_n < 0.0:
        i_n = 0.0
    if r_n < 0.0:
        r_n = 0.0

    scale = n / (s_n + i_n + r_n)
    return s_n * scale, i_n * scale, r_n * scale


def gen_sir(
    s: float,
    i: float,
    r: float,
    beta: float,
    gamma: float,
    n_days: int,
    x0: float = 0,
    k: float = 0,
    L: float = 1,
) -> Generator[Tuple[float, float, float], None, None]:
    """Simulate SIR model forward in time yielding tuples."""
    s, i, r  # = (float(v) for v in (s, i, r))
    n = s + i + r
    for d in range(n_days + 1):
        yield d, s, i, r
        bbeta = beta * (1 - logistic(d, L=L, x0=x0, k=k))
        s, i, r = sir(s, i, r, bbeta, gamma, n)


def sim_sir_df(
    s: float,
    i: float,
    r: float,
    beta: float,
    gamma: float,
    n_days: int,
    x0: float = 0,
    k: float = 0,
    L: float = 1,
) -> pd.DataFrame:
    """Simulate the SIR model forward in time."""
    return pd.DataFrame(
        data=gen_sir(s, i, r, beta, gamma, n_days, x0=x0, k=k, L=L),
        columns=("day", "susceptible", "infected", "recovered"),
    )


def get_dispositions(
    patients: np.ndarray, rates: Dict[str, float], market_share: float,
) -> Dict[str, np.ndarray]:
    """Get dispositions of patients adjusted by rate and market_share."""
    return {key: patients * rate * market_share for key, rate in rates.items()}


def build_admits_df(n_days, dispositions) -> pd.DataFrame:
    """Build admits dataframe from Parameters and Model."""
    days = np.arange(0, n_days + 1)
    projection = pd.DataFrame({"day": days, **dispositions,})
    # New cases
    admits_df = projection.iloc[:-1, :] - projection.shift(1)
    admits_df["day"] = range(admits_df.shape[0])
    return admits_df


def build_census_df(admits_df: pd.DataFrame, lengths_of_stay) -> pd.DataFrame:
    """ALOS for each category of COVID-19 case (total guesses)"""
    n_days = np.shape(admits_df)[0]
    census_dict = {}
    for key, los in lengths_of_stay.items():
        census = admits_df.cumsum().iloc[:-los, :] - admits_df.cumsum().shift(
            los
        ).fillna(
            0
        )  # .apply(np.ceil)
        census_dict[key] = census[key]

    census_df = pd.DataFrame(census_dict)
    census_df["day"] = census_df.index
    census_df = census_df[["day", *lengths_of_stay.keys()]]
    census_df = census_df.head(n_days)
    return census_df

In [33]:
MARKET_SHARE = 0.15
RATES = {
    "hospitalized": 0.025,
    "icu": 0.0075,
    "ventilated": 0.005,
}
LENGTH_OF_STAY = {
    "hospitalized": 7,
    "icu": 9,
    "ventilated": 10,
}

In [124]:
population_size = int(18.8e6)
recovery_days = 14.0 / 3.0
market_share = 1.0
initial_i = 100

prior = {
    "i": gv.gvar(initial_i, initial_i * 10.0),  # conservative 80% uncertainty
    # "r": gv.gvar(0.00, 0.01),  # no recovered to high prob
    "doubling time": gv.gvar(4.5, 3) / 3,  # doubling in 4.5 days +/- 3 days
    "gamma": 1 / gv.gvar(recovery_days, 3) * 3,  # recovery in 2 weeks +/- 3 days
    "k": gv.gvar(2, 1),
    "x0": gv.gvar(3, 2),
    "L": gv.gvar(0.8, 0.4),
}

In [125]:
def run_hosp_projections(n_days: int, p: Dict[str, "Gvar"]) -> "Array[Gvar]":
    """Runs sir for prior params and returns hospitalized 
    """
    # rr = p["r"]
    rr = 0
    S, I, R = population_size - p["i"] - rr, p["i"], rr

    intrinsic_growth_rate = 2 ** (1 / p["doubling time"]) - 1
    gamma = p["gamma"]
    beta = (
        (intrinsic_growth_rate + gamma)
        / S
        * (1 - logistic(0, L=1, x0=p["x0"], k=p["k"]))
    )

    nn_days = n_days - 1  # + LENGTH_OF_STAY["hospitalized"]

    raw_df = sim_sir_df(S, I, R, beta, gamma, nn_days, L=p["L"], x0=p["x0"], k=p["k"])

    i_dict_v = get_dispositions(raw_df.infected, RATES, MARKET_SHARE)
    r_dict_v = get_dispositions(raw_df.recovered, RATES, MARKET_SHARE)

    dispositions = {key: value + r_dict_v[key] for key, value in i_dict_v.items()}

    dispositions_df = pd.DataFrame(dispositions)
    admits_df = build_admits_df(nn_days, dispositions).drop(0)

    census_df = build_census_df(admits_df, LENGTH_OF_STAY).head(
        -LENGTH_OF_STAY["hospitalized"]
    )

    return raw_df.infected.values  # census_df["hospitalized"].values


def fcn(x: np.ndarray, p: Dict[str, "Gvar"]):
    """Wraps run_hosp_projections by truning days array to n_days.
    
    Should be updated later if time projection data is available.
    """
    return run_hosp_projections(len(x), p)


run_hosp_projections(15, prior)

array([100(1000), 158(1583), 248(2483), 364(3654), 398(4038), 286(2951),
       177(1863), 107(1154), 65(718), 39(449), 24(282), 14(177), 9(112),
       5(71), 3(45)], dtype=object)

In [126]:
X = nyc_hosp_df.index

In [127]:
ACTUAL_HOS = nyc_hosp_df.cases.values
P0 = 0.2
VARIANCE = ACTUAL_HOS * (1 - P0) * 4
Y = gv.gvar(ACTUAL_HOS, np.sqrt(VARIANCE))
Y

array([5.3(4.1), 23.7(8.7), 114(19), 774(50), 3270(102), 10243(181),
       19176(248), 27613(297), 37060(344), 43209(372)], dtype=object)

In [151]:
res = lsqfit.nonlinear_fit(data=(X, Y), fcn=fcn, prior=prior)

In [152]:
outputs = {key: val for key, val in res.p.items()}
inputs = dict(Y=Y)
inputs.update(prior)

print(
    res,
    "\n" * 2,
    gv.fmt_values(res.p),
    "\n" * 2,
    gv.fmt_errorbudget(outputs, inputs, percent=True),
)

Least Square Fit:
  chi2/dof [dof] = 2.1 [10]    Q = 0.02    logGBF = -91.405

Parameters:
              i    2.32 (96)     [ 100 (1000) ]  
  doubling time   0.330 (36)     [  1.5 (1.0) ]  *
          gamma    0.68 (41)     [  0.64 (41) ]  
              k   1.025 (98)     [  2.0 (1.0) ]  
             x0    3.01 (42)     [  3.0 (2.0) ]  
              L   0.886 (55)     [  0.80 (40) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 59/2.0)
  fitter = scipy_least_squares    method = trf
 

 Values:
                  i: 2.32(96)            
      doubling time: 0.330(36)           
              gamma: 0.68(41)            
                  k: 1.025(98)           
                 x0: 3.01(42)            
                  L: 0.886(55)           
 

 Partial % Errors:
                           i doubling time         gamma             k            x0             L
------------------------------------------------------------------------------------------

In [176]:
x_extrapolation = pd.date_range(
    nyc_hosp_df.index.min(), nyc_hosp_df.index.max() + timedelta(days=200), freq="3D"
)
y_fit = fcn(x_extrapolation, res.p)

In [190]:
fig = go.Figure()

y_min = gv.mean(y_fit) - gv.sdev(y_fit)
y_min = np.where(y_min > 0, y_min, 0)
fig.add_trace(
    go.Scatter(
        x=x_extrapolation, y=y_min, fill=None, mode="lines", line_color="indigo",
    )
)
fig.add_trace(
    go.Scatter(
        x=x_extrapolation,
        y=gv.mean(y_fit) + gv.sdev(y_fit),
        fill="tonexty",  # fill area between trace0 and trace1
        mode="lines",
        line_color="indigo",
    )
)

fig.add_trace(go.Scatter(x=nyc_hosp_df.index, y=gv.mean(Y), error_y_array=gv.sdev(Y)))

# fig.update_layout(yaxis_type="log")
# fig.update_layout(xaxis_range=(X.min(), X[9]))
fig.show()

In [191]:
yy = 1 - logistic(
    np.arange(len(x_extrapolation)), x0=res.p["x0"], k=res.p["k"], L=res.p["L"]
)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x_extrapolation,
        y=gv.mean(yy) - gv.sdev(yy),
        fill=None,
        mode="lines",
        line_color="indigo",
    )
)
fig.add_trace(
    go.Scatter(
        x=x_extrapolation,
        y=gv.mean(yy) + gv.sdev(yy),
        fill="tonexty",  # fill area between trace0 and trace1
        mode="lines",
        line_color="indigo",
    )
)
fig.update_layout(xaxis_range=(X.min(), X[9]))

fig.show()

In [181]:
def fitargs(z):
    dp = z
    pprior = {
        "i": gv.gvar(initial_i, initial_i * 10.0 * dp),  # conservative 80% uncertainty
        # "r": gv.gvar(0.00, 0.01),  # no recovered to high prob
        "doubling time": gv.gvar(4.5, 3 * dp) / 3,  # doubling in 4.5 days +/- 3 days
        "gamma": 1
        / gv.gvar(recovery_days, 3 * dp)
        * 3,  # recovery in 2 weeks +/- 3 days
        "k": gv.gvar(2, 1 * dp),
        "x0": gv.gvar(3, 2 * dp),
        "L": gv.gvar(0.8, 0.4 * dp),
    }
    return dict(prior=pprior, fcn=fcn, data=(X, Y))


fit, z = lsqfit.empbayes_fit(1.0, fitargs)

In [182]:
outputs = {key: val for key, val in res.p.items()}
inputs = dict(Y=Y)
inputs.update(prior)

print(
    fit,
    z,
    "\n" * 2,
    gv.fmt_values(fit.p),
    "\n" * 2,
    gv.fmt_errorbudget(outputs, inputs, percent=True),
)

Least Square Fit:
  chi2/dof [dof] = 2.4 [10]    Q = 0.0076    logGBF = -90.797

Parameters:
              i    2.46 (95)     [  100 (671) ]  
  doubling time   0.336 (34)     [  1.50 (67) ]  *
          gamma    0.67 (28)     [  0.64 (28) ]  
              k   1.043 (96)     [  2.00 (67) ]  *
             x0    3.08 (39)     [  3.0 (1.3) ]  
              L   0.883 (42)     [  0.80 (27) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 2/0.2)
  fitter = scipy_least_squares    method = trf
 0.6714843749999998 

 Values:
                  i: 2.46(95)            
      doubling time: 0.336(34)           
              gamma: 0.67(28)            
                  k: 1.043(96)           
                 x0: 3.08(39)            
                  L: 0.883(42)           
 

 Partial % Errors:
                           i doubling time         gamma             k            x0             L
---------------------------------------------------------------------